Imports the Firedrake library to initialize the simulation environment:

In [1]:
import firedrake

We import firedrake and set physical/numerical parameters: domain size (Lx, Lz), grid (Nx, Nz), sound speed c0, density rho, time step dt, total time t_end. smaller c0 and f0 slow the wave so you can observe it more clearly:

In [2]:
from firedrake import *
import math, time

Lx, Lz = 2000.0, 2000.0
Nx, Nz = 200, 200
c0     = 5.0
rho    = 1000.0
t_end  = 5.0
dt     = 0.01

f0     = 1.0
t0r    = 1.5/f0
amp    = 1.0

Defines the local output path, creates the necessary directories, and initializes the file writer:

In [3]:
import os
from firedrake import VTKFile

root = "." 
outdir = os.path.join(root, "outputs", "acoustic-out")
os.makedirs(outdir, exist_ok=True)
vtk = VTKFile(os.path.join(outdir, "acoustic.pvd"))

We create a 2D rectangular quadrilateral mesh. this is the physical domain for wave propagation:

In [4]:
mesh = RectangleMesh(Nx, Nz, Lx, Lz, quadrilateral=True)
x, z  = SpatialCoordinate(mesh)

Define function spaces: scalar V for pressure p, vector Vu for velocity v and displacement u. we also allocate functions for time states:

In [5]:
V  = FunctionSpace(mesh, "CG", 1)
Vu = VectorFunctionSpace(mesh, "CG", 1)

Q = TestFunction(V)
P = TrialFunction(V)

p     = Function(V,  name="p")
p_old = Function(V,  name="p_old")
v     = Function(Vu, name="v")
u     = Function(Vu, name="u")
speed = Function(V,  name="speed")

p.assign(0.0)
p_old.assign(0.0)
v.assign(Constant((0.0, 0.0)))
u.assign(Constant((0.0, 0.0)))

Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x749633f78350>, VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), name=None), Mesh(VectorElement(FiniteElement('Q', quadrilateral, 1), dim=2), 0)), 8)

We build a smooth spatial Gaussian (centered at (xs,z0)) multiplied by a temporal Ricker pulse:

In [6]:
xs   = 0.50*Lx
z0   = 0.50*Lz
sigx = 0.05*Lx
sigz = 0.05*Lz

src_shape = exp(-((x - xs)**2)/(2*sigx**2)) * exp(-((z - z0)**2)/(2*sigz**2))
S = Function(V, name="source")

def ricker(t):
    a = (math.pi*f0)**2
    tau = t - t0r
    return amp * (1.0 - 2.0*a*tau*tau) * math.exp(-a*tau*tau)

We use an implicit step for p:
$(m + d \sigma) p^{n+1} + c^2 Δ p^{n+1} = RHS$
with $m = 1 / \Delta t^2$, $d = 1 / Δ t$.

In [7]:
c2    = Constant(c0**2)
mcoef = Constant(1.0/dt**2)

Q = TestFunction(V)
P = TrialFunction(V)

a_form = (mcoef*P*Q + c2*dot(grad(P), grad(Q)))*dx
A = assemble(a_form)
solver = LinearSolver(A, solver_parameters={"ksp_type":"preonly", "pc_type":"lu"})
rhs = Function(V)

Write p, v, u, and |v| to a .pvd. optionally deform the mesh by u only when writing to better see displacements:

In [8]:
#vtk = VTKFile("acustica.pvd")
VIS_SCALE = 2.0
coords0 = mesh.coordinates.copy(deepcopy=True)

def write_out(k, every=5):
    if k % every != 0:
        return
    speed.interpolate(sqrt(inner(v, v)))
    if VIS_SCALE != 0.0:
        n = mesh.coordinates.dat.data.shape[0]
        mesh.coordinates.dat.data[:n, :] = coords0.dat.data_ro[:n, :] + VIS_SCALE*u.dat.data_ro[:n, :]
    vtk.write(p, v, u, speed)
    if VIS_SCALE != 0.0:
        mesh.coordinates.assign(coords0)

Temporal loop for updating p, v, and u:

In [9]:
steps = int(t_end/dt)
print(f"Running {steps} steps, dt={dt:.4f}s, c={c0} m/s")

t = 0.0
write_out(0, every=1)
t0_wall = time.time()
progress_every = max(1, steps//100)

for k in range(1, steps+1):
    t += dt

    S.interpolate(ricker(t) * src_shape)

    rhs_form = ((2.0*mcoef)*p - mcoef*p_old + S)*Q*dx
    b = assemble(rhs_form)

    p_new = Function(V)
    solver.solve(p_new, b)

    v.interpolate(v - (dt/rho)*grad(p_new))
    u.interpolate(u + dt*v)

    p_old.assign(p)
    p.assign(p_new)

    write_out(k, every=5)
    if (k % progress_every == 0) or (k == steps):
        elapsed = time.time() - t0_wall
        frac = k/steps
        total = elapsed/frac if frac > 0 else 0.0
        rem = max(0.0, total - elapsed)
        print(f"\r{k:6d}/{steps} ({100*frac:5.1f}%)  elapsed: {elapsed:6.1f}s  ETL: {rem:6.1f}s", end="")

Running 500 steps, dt=0.0100s, c=5.0 m/s
   500/500 (100.0%)  elapsed:  160.6s  ETL:    0.0s